In [58]:
import tweepy
import pandas as pd
import time
import re
import requests
import numpy as np

In [59]:
def key_txt():
    with open('keys.txt') as f:
        keys = [line.rstrip() for line in f]
    return keys

key_set = key_txt()

In [60]:
def key_init():
    auth = tweepy.OAuthHandler(key_set[0], key_set[1])
    auth.set_access_token(key_set[3], key_set[4])
    api = tweepy.API(auth)

In [61]:
username = "DcpNorthDelhi"
key_init()
failed = False
count = 0
try:
    count += 1
    mode = 'extended'
    tweets = tweepy.Cursor(api.user_timeline,id=username,tweet_mode = mode).items()
    tweet_list = []
    for tweet in tweets:
        tweets_list.append(tweet)
except BaseException as e:
    print('failed on_status,',str(e))
    if(failed):
        print("failed")
    time.sleep(3)

## Q1.

In [63]:

sources = []
sourceReply = {} 


def collect_data(mode):
    for tweet in tweets_list:
        if(str(tweet.in_reply_to_status_id_str) != "None" and count >= 0):
            try:
                source = api.get_status(tweet.in_reply_to_status_id,tweet_mode=mode)
                if username in source.full_text:
                    
                    sources.append(source)
                    sourceReply[source.id_str] = tweet.id_str
            except:
                pass
collect_data('extended')

### a.)

In [89]:
mail_regex = "([a-zA-Z0-9_\.-]+)@([\da-zA-Z\.-]+)\.([a-zA-Z\.]{2,6})"
aadhar_regex = "[2-9]{1}[0-9]{3}\\s[0-9]{4}\\s[0-9]{4}"
phone_regex = "\d{10}"
pincode_regex = "[1-2]{1}[0-9]{2}\s{0,1}[0-9]{3}"

def regex_support(regex, text):
    temp = re.findall(regex, text)
    return temp

dic = {}
dic['mails'] = []
dic['aadhars'] = []
dic['phones'] = []
dic['pincode'] = []

for s in sources:
    text = s.full_text
    dic['mails'] += (regex_support(mail_regex,text))
    dic['aadhars'] += (regex_support(aadhar_regex,text))
    dic['phones'] += (regex_support(phone_regex,text))
    dic['pincode'] += (regex_support(pincode_regex,text))

In [90]:

mails = set(dic['mails'])
aadhars = set(dic['aadhars'])
phones = set(dic['phones'])
pincodes = set(dic['pincode'])

print("phone ==> " , phones)
print("Pincodes => " , pincodes)

phone ==>  {'9811373344', '0816601007', '9196275118', '8527379140', '8178082002', '9899727721'}
Pincodes =>  {'110006', '234993', '210000', '219943', '116409', '273791', '178082', '196275', '110053', '166010', '110007', '113733', '229627', '106371', '136167', '230440', '110084', '110501', '113714'}


In [93]:


print("mail_count  => " ,len(mails))
print("aadhar_count  => " ,len(aadhars))
print("phone_count  => " ,len(phones))
print("pincode_count  => " ,len(pincodes))

mail_count  =>  0
aadhar_count  =>  0
phone_count  =>  6
pincode_count  =>  19


### b.)

In [95]:
def mediaextract(flag):
    mediaLinks = []
    count = 0
    for i in range(len(sources)):
        s = sources[i]
        count += 1
        if(flag):
            if ('media' in s.entities):
                mediaLinks.append(s.entities['media'][0]['media_url'])
    return mediaLinks

mediaLinks = set(mediaextract(True))
print(len(mediaLinks))


97


In [78]:
mediaCount = len(mediaLinks)

## Q2.

In [101]:
from datetime import datetime

tf = '%Y-%m-%d %H:%M:%S'

def reply_reponse(key, value):
    l = []
    l.append(api.get_status(int(key)).created_at)
    l.append(api.get_status(int(value)).created_at)
    responseTime = replyTime - sourceTime
    l.append(reponse_Time)
    return l
    

dates = []
responses = []

for key in sourceReply.keys():
    try:
        temp = reply_reponse(key, sourceReply[key])
        
        sourceTime = temp[0]
        replyTime = temp[1]
        responseTime = temp[2]

        dates.append(replyTime)
        responses.append(responseTime.total_seconds()/60)
    except:
        pass

In [102]:
res = [np.min(responses), np.max(responses) , np.mean(responses) , np.std(responses)]

print(" min Response Time => ",res[0])
print(" max Response Time => ",res[1])
print(" mean Response Time => ",res[2])
print(" std Response Time => ",res[3])

 min Response Time =>  4.183333333333334
 max Response Time =>  88485.21666666666
 mean Response Time =>  2104.7548484848485
 std Response Time =>  7690.824183865779


In [103]:
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure([go.Scatter(x=dates, y=responses)])

fig.update_layout(
    title="responses times(Time series)",
    xaxis_title="Date",
    yaxis_title="Response Time (in minutes)",
)

fig.show()